In [ ]:

import sklearn.cluster as cluster
import requests
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


from sklearn.cluster import DBSCAN

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation #For clustering
from sklearn.mixture import GaussianMixture #For GMM clustering

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
import numpy
import pandas

from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesRegressor


import matplotlib.pyplot as plt


from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error

In [ ]:
dataset = pd.read_csv('/kaggle/input/forest-fires-data-set/forestfires.csv')
dataset.head()

In [ ]:
del dataset["X"]
del dataset["Y"]
del dataset["month"]
del dataset["day"]
del dataset["FFMC"]
del dataset["DMC"]
del dataset["DC"]
del dataset["ISI"]


In [ ]:
dataset.head()

In [ ]:
corr = dataset.corr(method='pearson')
print("Correlation of the Dataset:",corr)

In [ ]:
data = dataset.values

X = data[:,0:4]
Y = data[:,4]
print(X)
print(Y)

In [ ]:
ETG = ExtraTreesRegressor(n_estimators = 100, criterion = 'mae')

In [ ]:
rfe = RFE(ETG, 5)

In [ ]:
fitting = rfe.fit(X, Y)
print("The number of features:", fitting.n_features_)
print("Selected Features:", fitting.support_)
print("Feature Rankings:", fitting.ranking_)

In [ ]:
print('Random Forest Regressor')
rfreg = RandomForestRegressor(n_estimators = 100)
rfreg.fit(X,Y)
prediction_rfreg = rfreg.predict(X)
score = explained_variance_score(Y, prediction_rfreg)
mae = mean_absolute_error(prediction_rfreg, Y)

print("Score:", score)
print("Mean Absolute Error:", mae)

In [ ]:
import pickle
from sklearn.externals import joblib 


In [ ]:
Pkl_Filename = "PredictingArea"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(rfreg, file)

In [ ]:
df = pd.read_csv("../input/modisdata/MODIS_C6_USA_contiguous_and_Hawaii_48h.csv")
df.head()


In [ ]:
df = df[df['confidence']>=60]
df.head()


In [ ]:
df = df[["latitude", "longitude"]]

In [ ]:
df.head()

In [ ]:
kmeans = KMeans(n_clusters=25)
labels =kmeans.fit(df)




In [ ]:
center = kmeans.cluster_centers_
print(center)
centers = []

In [ ]:
centers = list(center)
        

In [ ]:
centers
newCenters = []

In [ ]:
for i in centers:
    newCenters.append(i)

In [ ]:
fireX = np.array([[0,0,0,0]])

In [ ]:
for center in newCenters:
    lat = center[0]
    lng = center[1]
    url = "http://dataservice.accuweather.com/locations/v1/cities/geoposition/search"

    querystring = {"q":"{},{}".format(lat, lng), "apikey" : "3StNxANAsMCiWQwc4yptJsvamuwQkKF1"}
    response = requests.request("GET", url, params=querystring)
    response = response.json()
    key = response['Key']

    url = "http://dataservice.accuweather.com/currentconditions/v1/{}".format(key)
    querystring = {"apikey" : "3StNxANAsMCiWQwc4yptJsvamuwQkKF1","details":"true"}

    response = requests.request("GET", url, params=querystring).json()[0]

    windSpeed = (response['Wind']['Speed']['Metric']['Value'])
    temp = (response['Temperature']['Metric']['Value'])
    rain = (response['Precip1hr']['Metric']['Value'])
    humidity = (response['RelativeHumidity'])
    fireX = np.append(fireX, [[windSpeed, temp, rain, humidity]], axis = 0)
    print(fireX)

In [ ]:
fireX


In [ ]:
fireX = np.delete(fireX, (0), axis = 0)

In [ ]:
fireX

In [ ]:
with open(Pkl_Filename, 'rb') as file:  
    Pickled_LR_Model = pickle.load(file)

Pickled_LR_Model

In [ ]:
areas = Pickled_LR_Model.predict(fireX)
areas

In [ ]:
print(newCenters)

In [ ]:
fireResult = pd.DataFrame(data={'latitude/longitude': newCenters, 'area': areas})
print(fireResult.head())
fireResult.to_csv('fireResult.csv', index=False)
